In [1]:
import pennylane as qml
import pennylane.numpy as np

# symbols
symbols = ["Be", "H", "H"]

# geotry
a = 1.326 # Interatomic distance in Angstroms
geometry = np.array([[0.0, 0.0, 0.0],
            [a, 0.0, 0.0],
            [-a, 0.0, 0.0]])


# Generate the Hamiltonian
hamiltonian, qubits = qml.qchem.molecular_hamiltonian(
    symbols = symbols,
    coordinates= geometry,
    charge = 0,
    mult = 1,
    basis = 'sto-3g',
    active_electrons=4,
    active_orbitals=4,
    mapping='jordan_wigner'
)

print("Number of qubits needed:", qubits)
print("Hamiltonian:\n", hamiltonian)


Number of qubits needed: 8
Hamiltonian:
   (-13.766207984590432) [I0]
+ (0.01097121227559824) [Z4]
+ (0.010971212275598278) [Z5]
+ (0.01097121227561932) [Z6]
+ (0.010971212275619369) [Z7]
+ (0.1970465567145386) [Z2]
+ (0.1970465567145386) [Z3]
+ (0.20791827628045143) [Z1]
+ (0.2079182762804515) [Z0]
+ (0.08320325030862598) [Z0 Z2]
+ (0.08320325030862598) [Z1 Z3]
+ (0.08991776744427607) [Z2 Z4]
+ (0.08991776744427607) [Z3 Z5]
+ (0.08991776744429614) [Z2 Z6]
+ (0.08991776744429614) [Z3 Z7]
+ (0.09427773555506426) [Z4 Z6]
+ (0.09427773555506426) [Z5 Z7]
+ (0.09806500303531573) [Z0 Z4]
+ (0.09806500303531573) [Z1 Z5]
+ (0.09806500303533755) [Z0 Z6]
+ (0.09806500303533755) [Z1 Z7]
+ (0.10034008122331714) [Z4 Z7]
+ (0.10034008122331714) [Z5 Z6]
+ (0.10320339518218957) [Z0 Z5]
+ (0.10320339518218957) [Z1 Z4]
+ (0.10320339518221253) [Z0 Z7]
+ (0.10320339518221253) [Z1 Z6]
+ (0.10643107192612941) [Z2 Z5]
+ (0.10643107192612941) [Z3 Z4]
+ (0.10643107192615314) [Z2 Z7]
+ (0.10643107192615314) [Z3

In [6]:
from CQS.methods import *
#We will also use:
import numpy as np

from CQS.util.IO import tuplesToMatrix
import openfermion
from openfermion import FermionOperator

In [7]:
def convert_to_cqs(hamiltonian_terms, qubits):
    # Mapping from operator symbols to integers for CQS
    op_map = {'I': 0, 'X': 1, 'Y': 2, 'Z': 3}

    #Lists to hold the coefficients and operator tuples for each term
    coefs = []
    ops = []

    # Process each term in the Hamiltonian
    for term in hamiltonian_terms:
        coef, operators = term.strip().split(' ', 1)
        coef = float(coef.strip('()'))  # Strip parentheses before converting to float

    #Initialize operator tuple for this term with identity operators for all qubits
        op_tuple = [0] * qubits

        # Process each operator in the term
        if operators != '[]':  # Check if there are any operators
            # Remove brackets and split the operators
            op_list = operators[1:-1].split()
            for op in op_list:
                op_type, op_idx = op[0], int(op[1:])  # Extract operator type and index
                op_tuple[op_idx] = op_map[op_type]  # Map the operator to its integer representation

        # Append the coefficient and operator tuple to their respective lists
        coefs.append(coef)
        ops.append(tuple(op_tuple))

    return coefs, ops

str_hamiltonian=str(hamiltonian)

# Step 2: Split the string into individual terms
terms = str_hamiltonian.strip().split('\n+ ')

# Step 3 and 4: Trim whitespace and encapsulate each term in quotes
hamiltonian_terms = [f"{term.strip()}" for term in terms]

print(convert_to_cqs(hamiltonian_terms, qubits))



([-13.766207984590432, 0.01097121227559824, 0.010971212275598278, 0.01097121227561932, 0.010971212275619369, 0.1970465567145386, 0.1970465567145386, 0.20791827628045143, 0.2079182762804515, 0.08320325030862598, 0.08320325030862598, 0.08991776744427607, 0.08991776744427607, 0.08991776744429614, 0.08991776744429614, 0.09427773555506426, 0.09427773555506426, 0.09806500303531573, 0.09806500303531573, 0.09806500303533755, 0.09806500303533755, 0.10034008122331714, 0.10034008122331714, 0.10320339518218957, 0.10320339518218957, 0.10320339518221253, 0.10320339518221253, 0.10643107192612941, 0.10643107192612941, 0.10643107192615314, 0.10643107192615314, 0.1124647725597978, 0.11246477255984792, 0.11948395125516799, 0.12231333877548345, 0.12231333877548345, 0.13025646343899072, -0.03911008846685747, -0.03911008846685747, -0.016513304481857, -0.016513304481857, -0.016513304481853334, -0.016513304481853334, -0.006062345668252873, -0.006062345668252873, -0.005138392146874986, -0.005138392146874986, -

In [9]:
#Now, we can put all this together:

#Step 1: Create an Empty Hamiltonian Object
HubbardH = Hamiltonian(qubits)

#Use Hamiltonian.addTerms to build the Hubbard model Hamiltonian:
HubbardH.addTerms(convert_to_cqs(hamiltonian_terms, qubits))
#This gives:
HubbardH.getHamiltonian(type='printText')
#There's an IIII term we would rather not deal with, so we can remove it like this:
HubbardH.removeTerm((0,0,0,0,0,0,0,0))
#This gives:
print('Idenity/Global Phase removed:')
HubbardH.getHamiltonian(type='printText')

#Be careful choosing an involution, because it might now decompose such that the Hamiltonian is in M:
try:
    HubbardC = Cartan(HubbardH, involution='countX')
except Exception as e:
    print('Default Even/Odd Involution does not work:')
    print(e)
print('countY does work though. g = ')
#HubbardC = Cartan(HubbardH, involution='countY')
print(HubbardC.g)

-13.766207984590432 * IIIIIIII
0.01097121227559824 * IIIIZIII
0.010971212275598278 * IIIIIZII
0.01097121227561932 * IIIIIIZI
0.010971212275619369 * IIIIIIIZ
0.1970465567145386 * IIZIIIII
0.1970465567145386 * IIIZIIII
0.20791827628045143 * IZIIIIII
0.2079182762804515 * ZIIIIIII
0.08320325030862598 * ZIZIIIII
0.08320325030862598 * IZIZIIII
0.08991776744427607 * IIZIZIII
0.08991776744427607 * IIIZIZII
0.08991776744429614 * IIZIIIZI
0.08991776744429614 * IIIZIIIZ
0.09427773555506426 * IIIIZIZI
0.09427773555506426 * IIIIIZIZ
0.09806500303531573 * ZIIIZIII
0.09806500303531573 * IZIIIZII
0.09806500303533755 * ZIIIIIZI
0.09806500303533755 * IZIIIIIZ
0.10034008122331714 * IIIIZIIZ
0.10034008122331714 * IIIIIZZI
0.10320339518218957 * ZIIIIZII
0.10320339518218957 * IZIIZIII
0.10320339518221253 * ZIIIIIIZ
0.10320339518221253 * IZIIIIZI
0.10643107192612941 * IIZIIZII
0.10643107192612941 * IIIZZIII
0.10643107192615314 * IIZIIIIZ
0.10643107192615314 * IIIZIIZI
0.1124647725597978 * IIIIZZII
0.11246477

In [10]:
print(HubbardC.h)

[(0, 0, 0, 0, 3, 0, 0, 0), (0, 0, 0, 0, 0, 3, 0, 0), (0, 0, 0, 0, 0, 0, 3, 0), (0, 0, 0, 0, 0, 0, 0, 3), (0, 0, 3, 0, 0, 0, 0, 0), (0, 0, 0, 3, 0, 0, 0, 0), (0, 3, 0, 0, 0, 0, 0, 0), (3, 0, 0, 0, 0, 0, 0, 0), (3, 0, 3, 0, 0, 0, 0, 0), (0, 3, 0, 3, 0, 0, 0, 0), (0, 0, 3, 0, 3, 0, 0, 0), (0, 0, 0, 3, 0, 3, 0, 0), (0, 0, 3, 0, 0, 0, 3, 0), (0, 0, 0, 3, 0, 0, 0, 3), (0, 0, 0, 0, 3, 0, 3, 0), (0, 0, 0, 0, 0, 3, 0, 3), (3, 0, 0, 0, 3, 0, 0, 0), (0, 3, 0, 0, 0, 3, 0, 0), (3, 0, 0, 0, 0, 0, 3, 0), (0, 3, 0, 0, 0, 0, 0, 3), (0, 0, 0, 0, 3, 0, 0, 3), (0, 0, 0, 0, 0, 3, 3, 0), (3, 0, 0, 0, 0, 3, 0, 0), (0, 3, 0, 0, 3, 0, 0, 0), (3, 0, 0, 0, 0, 0, 0, 3), (0, 3, 0, 0, 0, 0, 3, 0), (0, 0, 3, 0, 0, 3, 0, 0), (0, 0, 0, 3, 3, 0, 0, 0), (0, 0, 3, 0, 0, 0, 0, 3), (0, 0, 0, 3, 0, 0, 3, 0), (0, 0, 0, 0, 3, 3, 0, 0), (0, 0, 0, 0, 0, 0, 3, 3), (0, 0, 3, 3, 0, 0, 0, 0), (3, 0, 0, 3, 0, 0, 0, 0), (0, 3, 3, 0, 0, 0, 0, 0), (3, 3, 0, 0, 0, 0, 0, 0), (3, 3, 0, 3, 0, 0, 0, 0), (0, 3, 3, 3, 0, 0, 0, 0), (3, 3, 3, 0

In [11]:
xyP = FindParameters(HubbardC)

KeyboardInterrupt: 